In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "..output_data/cities.csv"


# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)
weather_api_key

'd178fb16918d814296102796efe8a578'

## Generate Cities List

In [9]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination 
# using citipy module
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


632

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [10]:
#Init the lists
name=[]
lat=[]
max_temp=[]
lng=[]
humidity=[]
cloud=[]
wind_speed=[]
country=[]
date=[]
weather_data={}
#Build partial query URL
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
query_url = f"{url}appid={weather_api_key}&units={units}&q="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=d178fb16918d814296102796efe8a578&units=metric&q=


In [11]:
for city in cities : 
    try:
        response=requests.get(query_url + city).json()
        name.append(response['name']) 
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        cloud.append(response['clouds']['all'])
        wind_speed.append(response['wind']['speed'])
        country.append(response['sys']['country']) 
        date.append(response['dt'])
        count = cities.index(city)
        cname = response['name']
        print(f'Processing Record {count}: {cname}')
    except KeyError :
        print(f'City {city} not found')

Processing Record 0: Prokhorovka
Processing Record 1: Mataura
Processing Record 2: Port-Gentil
Processing Record 3: Vila Velha
City burkhala not found
Processing Record 5: Dikson
Processing Record 6: Hilo
Processing Record 7: Busselton
Processing Record 8: New Norfolk
Processing Record 9: Fortuna
Processing Record 10: Chapais
Processing Record 11: Puerto Escondido
Processing Record 12: Jalu
Processing Record 13: Cherskiy
Processing Record 14: Tomatlán
Processing Record 15: Krasnyy Klyuch
Processing Record 16: Kavieng
Processing Record 17: Dabakala
Processing Record 18: Vardø
Processing Record 19: Albany
Processing Record 20: Orlik
City taolanaro not found
Processing Record 22: Chippewa Falls
Processing Record 23: Caravelas
Processing Record 24: Punta Arenas
Processing Record 25: Atuona
Processing Record 26: Vaini
Processing Record 27: Ponta do Sol
Processing Record 28: Port Hardy
Processing Record 29: Broken Hill
Processing Record 30: Rikitea
Processing Record 31: Xining
Processing Rec

Processing Record 263: College
Processing Record 264: Eldorado
Processing Record 265: Ketchikan
City grand river south east not found
Processing Record 267: Matola
Processing Record 268: Saldanha
Processing Record 269: Sol'-Iletsk
Processing Record 270: Nyurba
Processing Record 271: Uporovo
Processing Record 272: Alice Springs
Processing Record 273: Bahía Blanca
Processing Record 274: Margate
Processing Record 275: Beira
Processing Record 276: Padang
Processing Record 277: Ahipara
Processing Record 278: Tiznit Province
Processing Record 279: Yangi Marg`ilon
Processing Record 280: Puro
Processing Record 281: Sechura
Processing Record 282: Qurayyat
City attawapiskat not found
Processing Record 284: Grindavik
Processing Record 285: Kildare
Processing Record 286: Castro
Processing Record 287: Mandan
Processing Record 288: Ribeira Grande
Processing Record 289: Puerto Carreño
Processing Record 290: Soto la Marina
Processing Record 291: Poya
Processing Record 292: Sambava
City vaitupu not fou

City bur gabo not found
Processing Record 522: Mortka
Processing Record 523: Pushkino
Processing Record 524: Kawalu
Processing Record 525: Birin
Processing Record 526: Punta Alta
Processing Record 527: Dwārka
Processing Record 528: Irricana
Processing Record 529: Ternate
Processing Record 530: Kalmunai
Processing Record 531: Auki
Processing Record 532: Arman'
City marcona not found
Processing Record 534: Kieta
Processing Record 535: Arrondissement de Lure
Processing Record 536: Vilhena
Processing Record 537: Tsiroanomandidy
Processing Record 538: Nanortalik
Processing Record 539: Husavik
Processing Record 540: Berbera
Processing Record 541: Damietta
City sentyabrskiy not found
Processing Record 543: Bogandé
Processing Record 544: Boralday
Processing Record 545: Port Macquarie
Processing Record 546: Guadalupe Victoria
Processing Record 547: Provincia di Campobasso
Processing Record 548: Kīhei
Processing Record 549: Laas
Processing Record 550: Babstovo
Processing Record 551: Manaure
Proc

In [7]:
weather_dict = {
    "city": name,
    "Lat": lat,
    "Lng" : lng,
    "Max temp": max_temp,
    "Humidity" : humidity,
    "Cloudiness" : cloud,
    "Wind speed" : wind_speed, 
    "Country" : country,
    "Date" : date
}


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [21]:
# city="grand gaube"
# unit="metric"
# query_url = f"{url}appid={weather_api_key}&units={unit}&q="
# test = response=requests.get(query_url + city).json()
weather_data = pd.DataFrame(weather_dict)
weather_data 

,city,Lat,Lng,Max temp,Humidity,Cloudiness,Wind speed,Country,Date
0,Tuktoyaktuk,69.45,-133.04,-31.00,74,75,3.10,CA,1607872862
1,Guerrero Negro,27.98,-114.06,15.01,64,0,5.07,MX,1607872862
2,Jamestown,42.10,-79.24,1.67,86,90,3.60,US,1607872592
3,Marabu,4.41,114.01,27.00,94,40,1.00,MY,1607872831
4,Albany,42.60,-73.97,11.67,80,37,0.45,US,1607872708
...,...,...,...,...,...,...,...,...,...
543,Arandis,-22.42,14.97,18.33,80,0,6.68,NA,1607872903
544,Honningsvåg,70.98,25.97,1.73,83,95,6.62,NO,1607872903
545,Rio Grande,-32.03,-52.10,28.33,78,20,4.10,BR,1607872903
546,Nizhnyaya Tura,58.63,59.81,-9.65,86,94,5.85,RU,1607872903


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression